In [8]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [10]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|                 Fun|It is a fun game ...| 2015-08-31|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...|     Mobile_Apps|          5|    

In [11]:
# filter dataset fot total_votes greater than or equal to 20
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vote_count = df.filter("total_votes >= 20")

In [12]:
# create a dataframe to retrieve all rows where the number of helpful_votes/total_vote >= 50%
votes_df = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"] >= 0.5)
votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10869226|R1Z9QZRQJ1F8JX|B004GJDQT8|      36653526|  Amazon Underground|     Mobile_Apps|          1|           17|         22|   N|                Y|These permissions...|Why not ask for a...| 2015-08-31|
|         US|    6059356|R31PAYBVBP8W62|B012OO01XK|      19786981| Monsters University|     Mobile_Apps|          1|    

In [13]:
# create a dataframe to retrieve all rows where the review was written as part of the Vine program (paid) where vine == 'Y'
votes_df.filter(votes_df["vine"] == "Y").show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52490988|R3BOQTLUU3Y4L8|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|          4|           42|         55|   Y|                N|Lot of fun, but f...|Before receiving ...| 2013-11-19|
|         US|   48814959| RWF03LXVXC22A|B00EP0SIOG|     628459233|Orbotix Sphero 2....|Mobile_Electronics|      

In [14]:
# create a dataframe to retrieve all rows where the review was not written as part of the Vine program (paid) where vine == 'N'
votes_df.filter(votes_df["vine"] == "N").show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10869226|R1Z9QZRQJ1F8JX|B004GJDQT8|      36653526|  Amazon Underground|     Mobile_Apps|          1|           17|         22|   N|                Y|These permissions...|Why not ask for a...| 2015-08-31|
|         US|    6059356|R31PAYBVBP8W62|B012OO01XK|      19786981| Monsters University|     Mobile_Apps|          1|    

In [15]:
# total number of 5 star reviews
five_star_reviews = votes_df.filter(votes_df["star_rating"] == 5)
five_star_reviews.count()

59806

In [16]:
# total number of reviews
votes_df.count()

130584

In [17]:
# percentage of 5 star reviews
five_star_reviews.count() / votes_df.count()

0.45798872756233533

In [18]:
# total number of reviews for paid
votes_df.filter(votes_df["verified_purchase"] == "Y").count()

117221

In [19]:
# total number of 5 star reviews for paid
five_star_reviews.filter(five_star_reviews["verified_purchase"] == "Y").count()

54632

In [20]:
# percentage of paid 5 star reviews
five_star_reviews.filter(five_star_reviews["verified_purchase"] == "Y").count() / votes_df.filter(votes_df["verified_purchase"] == "Y").count()

0.46605983569496934

In [21]:
# total number of reviews for unpaid
votes_df.filter(votes_df["verified_purchase"] == "N").count()

13363

In [22]:
# total number of 5 star reviews for unpaid
five_star_reviews.filter(five_star_reviews["verified_purchase"] == "N").count()

5174

In [23]:
# percentage of unpaid 5 star reviews
five_star_reviews.filter(five_star_reviews["verified_purchase"] == "N").count() / votes_df.filter(votes_df["verified_purchase"] == "N").count()

0.38718850557509543